In [57]:
import numpy as np
import pandas as pd
!pip install emoji
import emoji
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,SimpleRNN,Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam


In [19]:
df=pd.read_csv(r'/content/emoji_data.csv',header=None)
df.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [20]:
emoji.emojize(":red_heart:")

'❤️'

In [21]:
emoji_dict={
    0:":red_heart:",
    1:":baseball:",
    2:":grinning_face_with_big_eyes:",
    3:":disappointed_face:",
    4:":fork_and_knife_with_plate:"
}
def label_to_emoji(label):
  return emoji.emojize(emoji_dict[label])
X=df[0].values
y=df[1].values


In [39]:
#embedding
print(y)
y[y=='0v2']='0'

['4' '3' '3 ' '1 ' '2' '1' '4' '3' '4' '1' '3' '3 ' '2' '2' '4' '3' '2'
 '3 ' '3 ' '1' '3 ' '2' '2' '2' '0' '1' '0' '4 ' '2' '0v2' '2' '0' '0'
 '3 ' '4' '0' '2' '1' '3' '1' '0' '4' '0 ' '3' '0 ' '4' '2' '3 ' '4' '2 '
 '2' '3' '0' '2' '2' '3 ' '2' '3' '2' '2' '3 ' '3' '0 ' '2' '3' '0' '2'
 '0' '0 ' '2' '3' '2' '4' '1' '3' '3' '0' '0' '3' '2' '0' '3' '0' '2' '2'
 '4' '2' '2' '0' '0' '2' '3' '0' '4' '2' '1' '2' '3' '3' '2' '3' '0' '3'
 '0' '2' '0' '2' '3' '4' '3' '1' '3' '4' '3' '2' '3' '3' '3' '1' '4' '4'
 '2' '2' '1' '1' '2' '3' '2' '3' '4' '2' '3' '0' '2' '0' '0' '4' '3' '4'
 '2' '3' '2' '3' '4' '2' '1' '2' '4' '3' '1' '3' '2' '3' '2' '2' '3' '3'
 '2' '4' '0' '0' '0' '3' '0' '0' '1' '1' '2' '2' '2' '0' '3' '2' '3' '3'
 '1' '2' '2' '4' '2' '3' '1' '2']


In [23]:
file=open('glove.6B.100d.txt','r',encoding='utf8')
content=file.readlines()
file.close()
#prints every word with its 100 dimensional vector encoding


In [45]:
#content
embeddings={}
for line in content:
  line=line.split()
  embeddings[line[0]]=np.array(line[1:],dtype=np.float32)

KeyError: 'macaroon'

In [26]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X)
word2index=tokenizer.word_index

In [29]:
x_tokens=tokenizer.texts_to_sequences(X)
def get_maxlen(data):
  maxlen=0
  for line in data:
    maxlen=max(maxlen,len(line))
  return maxlen
maxlen=get_maxlen(x_tokens)
print(maxlen)

10


In [35]:
x_train=pad_sequences(x_tokens,padding='post',truncating='post')

In [40]:
y_train=to_categorical(y)

In [47]:
#model
embed_size=100
embedding_maxtrix=np.zeros((len(word2index)+1,embed_size))
for word,i in word2index.items():
  embed_vector=embeddings.get(word)
  embedding_maxtrix[i]=embed_vector
embedding_maxtrix


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.046539  ,  0.61966002,  0.56647003, ..., -0.37616   ,
        -0.032502  ,  0.80620003],
       [-0.49886   ,  0.76602   ,  0.89750999, ..., -0.41179001,
         0.40538999,  0.78504002],
       ...,
       [-0.46263   ,  0.069864  ,  0.69094998, ..., -0.29174   ,
         0.32041001,  0.21201999],
       [ 0.073242  ,  0.11134   ,  0.62281001, ...,  0.53416997,
        -0.1646    , -0.27516001],
       [ 0.29019001,  0.80497003,  0.31187001, ..., -0.33603001,
         0.45998001, -0.11278   ]])

In [60]:
model=Sequential(
    [Embedding(input_dim=len(word2index)+1,output_dim=embed_size,input_length=maxlen,weights=[embedding_maxtrix],trainable=False),
     LSTM(units=16,return_sequences=True),
    LSTM(units=4),
     Dense(5,activation='softmax')
    ]
)
model.compile(optimizer=Adam(learning_rate=0.00001),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=120)

Epoch 1/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.1604 - loss: nan
Epoch 2/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1839 - loss: nan
Epoch 3/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1735 - loss: nan
Epoch 4/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1606 - loss: nan
Epoch 5/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1949 - loss: nan
Epoch 6/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1646 - loss: nan
Epoch 7/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1682 - loss: nan
Epoch 8/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2077 - loss: nan
Epoch 9/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1820 - loss: nan
Epoch 10/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2182 - loss: nan
Epoch 11/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1732 - loss: nan
Epoch 12/120
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.1843 - loss: nan
Epoch 13/120
6/6 ━━━━━━━━

In [52]:
model.predict(np.ones((1,maxlen))).shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step


(1, 16)